In [1]:
!nvidia-smi

Thu May 12 13:43:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:D8:00.0 Off |                    0 |
| N/A   38C    P0    70W / 250W |      0MiB / 45556MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
path_to_json_file = 'News_dataset/Sarcasm_Headlines_Dataset_v2.json'

In [3]:
import json 

with open(path_to_json_file, 'r') as j:
     json_data = [json.loads(line) for line in j]

In [4]:
import random 
import numpy as np
import torch

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [5]:
import json 
di = dict()

i=0
for headline in json_data:
    d1 = {'text': headline['headline'], 'label': headline['is_sarcastic']}

    di[i] = d1
    i+=1

In [6]:
from transformers import CanineTokenizer, CanineForSequenceClassification

tokenizer = CanineTokenizer.from_pretrained("google/canine-s")

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [7]:
tokenized_datasets = []
for i in di:
  l = tokenizer(di[i]["text"], padding="max_length", truncation=True)
  l['labels'] = di[i]["label"]
  tokenized_datasets.append(l)

In [8]:
random.shuffle(tokenized_datasets)

train_data = tokenized_datasets[:17171]
test_data = tokenized_datasets[17171:22895]
val_data = tokenized_datasets[22895:]

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
model = CanineForSequenceClassification.from_pretrained("google/canine-s", num_labels=2)

Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)


    return {"accuracy": accuracy}

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer/new_ep/news/8_ep",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/home/mss9240/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17171
  Num Epochs = 8
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8592


Epoch,Training Loss,Validation Loss,Accuracy
1,0.448500,0.356031,0.848532
2,0.339000,0.311424,0.870370
3,0.262300,0.356483,0.863033
4,0.201300,0.377556,0.887491
5,0.165400,0.521560,0.878407
6,0.142700,0.510373,0.891684
7,0.111200,0.571992,0.889762
8,0.099400,0.571963,0.890112


***** Running Evaluation *****
  Num examples = 5724
  Batch size = 16
Saving model checkpoint to test_trainer/new_ep/news/8_ep/checkpoint-1074
Configuration saved in test_trainer/new_ep/news/8_ep/checkpoint-1074/config.json
Model weights saved in test_trainer/new_ep/news/8_ep/checkpoint-1074/pytorch_model.bin
tokenizer config file saved in test_trainer/new_ep/news/8_ep/checkpoint-1074/tokenizer_config.json
Special tokens file saved in test_trainer/new_ep/news/8_ep/checkpoint-1074/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5724
  Batch size = 16
Saving model checkpoint to test_trainer/new_ep/news/8_ep/checkpoint-2148
Configuration saved in test_trainer/new_ep/news/8_ep/checkpoint-2148/config.json
Model weights saved in test_trainer/new_ep/news/8_ep/checkpoint-2148/pytorch_model.bin
tokenizer config file saved in test_trainer/new_ep/news/8_ep/checkpoint-2148/tokenizer_config.json
Special tokens file saved in test_trainer/new_ep/news/8_ep/checkpoint-2148/spec

TrainOutput(global_step=8592, training_loss=0.23356557014490195, metrics={'train_runtime': 9678.0912, 'train_samples_per_second': 14.194, 'train_steps_per_second': 0.888, 'total_flos': 1.8045559890935808e+17, 'train_loss': 0.23356557014490195, 'epoch': 8.0})

In [13]:
trainer.evaluate(eval_dataset = test_data)

***** Running Evaluation *****
  Num examples = 5724
  Batch size = 16


{'eval_loss': 0.629116952419281,
 'eval_accuracy': 0.8801537386443047,
 'eval_runtime': 150.7367,
 'eval_samples_per_second': 37.974,
 'eval_steps_per_second': 2.375,
 'epoch': 8.0}

In [14]:
trainer.save_model("test_trainer/new_ep/news/8_ep/model")

Saving model checkpoint to test_trainer/new_ep/news/8_ep/model
Configuration saved in test_trainer/new_ep/news/8_ep/model/config.json
Model weights saved in test_trainer/new_ep/news/8_ep/model/pytorch_model.bin
tokenizer config file saved in test_trainer/new_ep/news/8_ep/model/tokenizer_config.json
Special tokens file saved in test_trainer/new_ep/news/8_ep/model/special_tokens_map.json


In [16]:
import os
for filename in os.listdir("test_trainer/new_ep/news/8_ep"):
    print (filename)
    if filename == "runs":
        continue
    model_path = "test_trainer/new_ep/news/8_ep/"+filename
    model = CanineForSequenceClassification.from_pretrained(model_path, num_labels=2)



    test_trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=val_data,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    print(test_trainer.evaluate(eval_dataset = test_data))

loading configuration file test_trainer/new_ep/news/8_ep/checkpoint-2148/config.json
Model config CanineConfig {
  "_name_or_path": "google/canine-s",
  "architectures": [
    "CanineForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 57344,
  "downsampling_rate": 4,
  "eos_token_id": 57345,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "local_transformer_stride": 128,
  "max_position_embeddings": 16384,
  "model_type": "canine",
  "num_attention_heads": 12,
  "num_hash_buckets": 16384,
  "num_hash_functions": 8,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 16,
  "upsampling_kernel_size": 4,
  "use_cache": true
}

loading weights file test_trainer/new_ep/news/8_ep/checkpoint-2148/pytorch_model.bin


checkpoint-2148


All model checkpoint weights were used when initializing CanineForSequenceClassification.

All the weights of CanineForSequenceClassification were initialized from the model checkpoint at test_trainer/new_ep/news/8_ep/checkpoint-2148.
If your task is similar to the task the model of the checkpoint was trained on, you can already use CanineForSequenceClassification for predictions without further training.
***** Running Evaluation *****
  Num examples = 5724
  Batch size = 16


loading configuration file test_trainer/new_ep/news/8_ep/checkpoint-5370/config.json
Model config CanineConfig {
  "_name_or_path": "google/canine-s",
  "architectures": [
    "CanineForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 57344,
  "downsampling_rate": 4,
  "eos_token_id": 57345,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "local_transformer_stride": 128,
  "max_position_embeddings": 16384,
  "model_type": "canine",
  "num_attention_heads": 12,
  "num_hash_buckets": 16384,
  "num_hash_functions": 8,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 16,
  "upsampling_kernel_size": 4,
  "use_cache": true
}

loading weights file test_trainer/new_ep/news/8_ep/checkpoint-5370/pytorch_model.bin


{'eval_loss': 0.3223313093185425, 'eval_accuracy': 0.8570929419986024, 'eval_runtime': 148.9493, 'eval_samples_per_second': 38.429, 'eval_steps_per_second': 2.404}
checkpoint-5370


...

All model checkpoint weights were used when initializing CanineForSequenceClassification.

All the weights of CanineForSequenceClassification were initialized from the model checkpoint at test_trainer/new_ep/news/8_ep/checkpoint-7518.
If your task is similar to the task the model of the checkpoint was trained on, you can already use CanineForSequenceClassification for predictions without further training.
***** Running Evaluation *****
  Num examples = 5724
  Batch size = 16


loading configuration file test_trainer/new_ep/news/8_ep/checkpoint-3222/config.json
Model config CanineConfig {
  "_name_or_path": "google/canine-s",
  "architectures": [
    "CanineForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 57344,
  "downsampling_rate": 4,
  "eos_token_id": 57345,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "local_transformer_stride": 128,
  "max_position_embeddings": 16384,
  "model_type": "canine",
  "num_attention_heads": 12,
  "num_hash_buckets": 16384,
  "num_hash_functions": 8,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 16,
  "upsampling_kernel_size": 4,
  "use_cache": true
}

loading weights file test_trainer/new_ep/news/8_ep/checkpoint-3222/pytorch_model.bin


{'eval_loss': 0.6217502355575562, 'eval_accuracy': 0.8824248777078966, 'eval_runtime': 148.3326, 'eval_samples_per_second': 38.589, 'eval_steps_per_second': 2.413}
checkpoint-3222


All model checkpoint weights were used when initializing CanineForSequenceClassification.

All the weights of CanineForSequenceClassification were initialized from the model checkpoint at test_trainer/new_ep/news/8_ep/checkpoint-3222.
If your task is similar to the task the model of the checkpoint was trained on, you can already use CanineForSequenceClassification for predictions without further training.
***** Running Evaluation *****
  Num examples = 5724
  Batch size = 16


{'eval_loss': 0.3850793242454529, 'eval_accuracy': 0.8560447239692522, 'eval_runtime': 148.2984, 'eval_samples_per_second': 38.598, 'eval_steps_per_second': 2.414}


In [18]:
model_path = "test_trainer/new_ep/news/8_ep/checkpoint-6444"
model = CanineForSequenceClassification.from_pretrained(model_path, num_labels=2)



test_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_trainer.evaluate(eval_dataset = test_data)

loading configuration file test_trainer/new_ep/news/8_ep/checkpoint-6444/config.json
Model config CanineConfig {
  "_name_or_path": "google/canine-s",
  "architectures": [
    "CanineForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 57344,
  "downsampling_rate": 4,
  "eos_token_id": 57345,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "local_transformer_stride": 128,
  "max_position_embeddings": 16384,
  "model_type": "canine",
  "num_attention_heads": 12,
  "num_hash_buckets": 16384,
  "num_hash_functions": 8,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 16,
  "upsampling_kernel_size": 4,
  "use_cache": true
}

loading weights file test_trainer/new_ep/news/8_ep/checkpoint-6444/pytorch_model.bin
Al

{'eval_loss': 0.5514761805534363,
 'eval_accuracy': 0.88277428371768,
 'eval_runtime': 146.903,
 'eval_samples_per_second': 38.964,
 'eval_steps_per_second': 2.437}